**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Mar 28 17:54:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 3E50E41B-812A-4DEB-8305-843D59DD3C7A.jpeg
 6C10DC3D-9407-4CF2-B150-BCF986EE5C7B.jpeg
 achievements
 aia
'APP 03 VIDEO EDIT.mp4'
'CamScanner 04-18-2020 22.23.34.pdf'
'C notes'
'C notes '
'C++notes '
'Colab Notebooks'
'COM014040(2).pdf'
'Dad function '
'Datapoem assignment.gdoc'
'Document Studio'
'face emotion'
'File Encryption paper_May 9.pdf'
'final hack 2020 (1).pdf'
'final submtion.docx'
'Google Photos'
'How to get started with Drive.pdf'
'https:  goo.gl .txt'
'ICRTEEC2021_paper 80_Ambulance Booking System_10_Oct.pdf'
 ICRTEEC2021_Paper_ID_080.docx
'ICRTEEC - 2021 pid-080.docx'
 image.gsheet
 images.zip
 IMG_20150905_174723899.jpg
 IMG_20150917_090411038.jpg
 IMG_20151022_114051612.jpg
 IMG_20151023_125533700.jpg
 IMG_20151026_185927089.jpg
 IMG_20151026_203508622.jpg
 IMG_20151103_174500424.jpg
 IMG_20151111_211721539.jpg
 IMG_20151230_192808695.jpg
 IMG_20160101_192430338.jpg
 IMG_20160101_192504139.jpg
 IMG_20160131_172718435.jpg
 IMG_20160209_100215.jp

**1) Clone, configure & compile Darknet**



In [4]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15833, done.
remote: Total 15833 (delta 0), reused 0 (delta 0), pack-reused 15833
Receiving objects: 100% (15833/15833), 14.35 MiB | 12.76 MiB/s, done.
Resolving deltas: 100% (10670/10670), done.


In [5]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [6]:
# Compile
!make

mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variable ‘bu

**2) Configure yolov3.cfg file**

In [7]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [8]:
# Change lines in yolov3.cfg file
# 15 is base 4class*3 =n 27
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=4@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=4@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=4@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=27@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=27@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=27@' cfg/yolov3_training.cfg

**3) Create .names and .data files**

In [9]:
!echo -e '0\n1\n2\n3' > data/obj.names
!echo -e 'classes= 4\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [10]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

**5) Create a folder and unzip image dataset**

In [11]:
!mkdir data/obj
!unzip /mydrive/images.zip -d data/obj

Streaming output truncated to the last 5000 lines.
  inflating: data/obj/Apple___Cedar_apple_rust 272.txt  
  inflating: data/obj/Apple___Cedar_apple_rust 273.jpg  
  inflating: data/obj/Apple___Cedar_apple_rust 273.txt  
  inflating: data/obj/Apple___Cedar_apple_rust 274.jpg  
  inflating: data/obj/Apple___Cedar_apple_rust 274.txt  
  inflating: data/obj/Apple___Cedar_apple_rust 275.jpg  
  inflating: data/obj/Apple___Cedar_apple_rust 275.txt  
  inflating: data/obj/Apple___Cedar_apple_rust 276.jpg  
  inflating: data/obj/Apple___Cedar_apple_rust 276.txt  
  inflating: data/obj/Apple___Cedar_apple_rust 277.jpg  
  inflating: data/obj/Apple___Cedar_apple_rust 277.txt  
  inflating: data/obj/Apple___Cedar_apple_rust 278.jpg  
  inflating: data/obj/Apple___Cedar_apple_rust 278.txt  
  inflating: data/obj/Apple___Cedar_apple_rust 279.jpg  
  inflating: data/obj/Apple___Cedar_apple_rust 279.txt  
  inflating: data/obj/Apple___Cedar_apple_rust 28.jpg  
  inflating: data/obj/Apple___Cedar_ap

**6) Create train.txt file**

In [12]:
import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

**7) Download pre-trained weights for the convolutional layers file**

In [14]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2024-03-28 18:01:53--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  48.6MB/s    in 3.4s    

2024-03-28 18:01:57 (45.9 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**8) Start training**

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, GPU count: 1  
 OpenCV version: 4.5.4
yolov3_training
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Sho